# Études pour le model 1 :

Notre étude va se dérouler en plusieurs parties. 

1. Mise en place d'un modèle de base :

    **pre_traitement()**

    Entrée : nos csv
    
    Sortie : Une matrice contenant les données que nous pensons utile pour ce modèle après avoir enlevé les stop_words, la ponctuation et les majuscules.

    
    **Choix d'un modèle de base**

    **genere_embedding()**

    Entrée : la sortie de pre_traitement()
    
    Sortie : stock dans un fichier les embeddings correspondant à nos solutions.

    **find_solution()**

    Entrée : secteur, description

    Sortie : Une liste des meilleures solutions classés dans l'orde. On peut imaginer mettre une limite par exemple distance cos > 0.65

    **Tester conso dans gaia**

2. Creation d'un Test set 

    **Creation du test_set**

    On génère des description et domaine d'activité en fonction des infos de solution avec GPT4. On génère différent niveau de précision par solution.

    **test_model()**

    Entrée : find_solution() de notre modèle et test_set.

    Sortie : On renvoie une accuracy. On considère que si le modèle prédit la bonne solution dans son top 3 sol, c'est validé.

3. Test de différents modèles. L'idée est d'avoir un tableau avec la conso, l'accuracy et le model pour expliquer pourquoi on a pris celui-ci plutôt qu'un autre.

    **Faire un tableau de tests**
    
    Tester différents modèles et type de modèles, Word2Vec, FastText, SentenceTransformers (plusieurs).
    
    **Test Fine tuning**
    
    Fine tunner le meilleur pour voir le résultat.

    **Si temps, on peut même tenter d'entrainer un modèle sur notre test set avec notre méthode**



# 1. Mise en place d'un modèle de base

## Chargement des données

Selection données à utiliser et création dataset :

Ici on souhaite récupérer toutes les données pertinentes pour nos solutions et les concaténer afin d'avoir un dataset : [num_sol, données_pertinentes]

Voir si on prend plus de données par la suite en ajoutant les technologies.

In [30]:
!pip install pandas

In [32]:
import pandas as pd

# Charger le fichier CSV en spécifiant le séparateur '|'
df = pd.read_csv('../data/solutions.csv', sep='|', header=None)

# Renommer les colonnes
df.columns = ['id_solution', 'categorie', 'texte']

# Filtrer les lignes pour les catégories spécifiées
categories_specifiees = [1, 2, 5, 6]
df_filtre = df[df['categorie'].isin(categories_specifiees)]

# Pivoter les données pour obtenir les colonnes 'titre', 'definition', 'application' et 'bilan énergie'
df_pivot = df_filtre.pivot(index='id_solution', columns='categorie', values='texte').reset_index()

# Renommer les colonnes
df_pivot.columns = ['id_solution', 'titre', 'definition', 'application', 'bilan_energie']

# Afficher le dataset final
# print(df_pivot)

In [33]:
# Ajouter un point à la fin des colonnes si nécessaire
colonnes = ['titre', 'definition', 'application', 'bilan_energie']
for col in colonnes:
    df_pivot[col] = df_pivot[col].apply(lambda x: x.strip() + '.' if isinstance(x, str) and x.strip()[-1] != '.' else x.strip() if isinstance(x, str) else x)

# Fusionner les colonnes en une seule colonne 'text'
df_pivot['text'] = df_pivot['titre'] + ' ' + df_pivot['definition'] + ' ' + df_pivot['application'].astype(str) + ' ' + df_pivot['bilan_energie']

# Sélectionner uniquement les colonnes 'id_solution' et 'text'
df_final = df_pivot[['id_solution', 'text']]

# Afficher le DataFrame final
# print(df_final)

In [66]:
# Configure Pandas pour afficher le contenu complet des cellules
pd.set_option('display.max_colwidth', None)

# Afficher la solution avec l'ID 2
solution_id = df_final[df_final['id_solution'] == 54]
print(solution_id)

    id_solution  \
48           54   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

## Pré-traitement et nettoyage des données

In [46]:
!pip install spacy bs4

  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached Jinja2-3.1.3-py3-none-any.whl.metadata (3.3 kB)
  Using cached charset_normalizer-3.3.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (33 kB)
  Using cached urllib3-2.2.1-py3-none-any.whl.metadata (6.4 kB)
  Using cached certifi-2024.2.2-py3-none-any.whl.metadata (2.2 kB)
  Using cached MarkupSafe-2.1.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 6.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.6/181.6 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.2/157.2 kB 5.9 MB/s eta 0:00:00
Using cached requests-2.31.0-py3-none-any.whl (62 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 490.9/490.9 kB 

In [71]:
import re
import unicodedata
import spacy
from bs4 import BeautifulSoup
from spacy.lang.fr.stop_words import STOP_WORDS

# Charger le modèle spaCy pour le français
nlp = spacy.load("fr_core_news_sm")

def pre_processing(texte):
    # Nettoyer HTML Tags
    texte = BeautifulSoup(texte, 'html.parser').get_text()

    # Remplacer "&nbsp;." par rien
    texte = re.sub(r'&nbsp;\.', '', texte)

    # Accents
    texte = unicodedata.normalize('NFD', texte).encode('ascii', 'ignore').decode("utf-8")

    # Retirer les numéros
    texte = re.sub(r'\b\d+\b', '', texte)

    # Tokenization, Lemmatization, Removing Stopwords, Lowercase
    doc = nlp(texte)
    phrases_propres = []
    for phrase in doc.sents:
        tokens = [token.lemma_.lower() for token in phrase if not token.is_stop and not token.is_punct and not token.is_space]
        phrase_propre = ' '.join(tokens)
        if phrase_propre:
            phrases_propres.append(phrase_propre + ".")  # Ajouter un point à la fin de la phrase propre

    # Joining the cleaned sentences back into a single string
    cleaned_text = ' '.join(phrases_propres)

    return cleaned_text

# Exemple d'utilisation
texte_original = "Ce texte contient des &nbsp;. &nbsp;. balises HTML, des caractères spéciaux comme des accents ééèà, des chiffres 123, et de la ponctuation ! ? Cette phrase a un point. Et une autre test aussi."
texte_pre_traite = pre_processing(texte_original)
print("Texte original:")
print(texte_original)
print("\nTexte pré-traité:")
print(texte_pre_traite)


Texte original:
Ce texte contient des &nbsp;. &nbsp;. balises HTML, des caractères spéciaux comme des accents ééèà, des chiffres 123, et de la ponctuation ! ? Cette phrase a un point. Et une autre test aussi.

Texte pré-traité:
texte contenir. balise html caractere special accent eeeer chiffre ponctuation. phrase point. test.


In [72]:
text_solution = solution_id['text'].iloc[0]
print(text_solution)

text_pre_traite = pre_processing(text_solution)
print(text_pre_traite)

Moteur à haut rendement. Il existe plusieurs catégories de performance des moteurs selon leur rendement, qui s'appellent IE1 et IE2 en Europe. Depuis mi 2011, les nouveaux moteurs doivent être au minimum de classe IE2. Les constructeurs développent des moteurs répondant aux classes IE3 (obligatoire à partir de 2015 et 2017) et IE4 (Super Premium). Comparés aux moteurs standards, les moteurs à haut rendement ont été travaillés pour diminuer les pertes internes aux moteurs. Selon les classes de performance, cette réduction peut aller jusqu’à 45 %. Hors rupture de technologie, les moteurs à haut rendement sont de même taille et ont une augmentation de poids d’environ 15 %. Plus la puissance du moteur est faible, plus le gain sur le rendement du moteur est important. Pour autant, l’utilisation de moteurs à haut rendement sur de fortes puissances est aussi intéressante car, si le pourcentage d’augmentation est plus faible, il concerne une puissance plus importante. Sur la durée de vie total

## Implémentation Sentence Transformers

### Choix du modèle

### Génération et sauvegarde des embeddings

### Test find_solution()

## 2. Génération test set 

En utilisant GPT4, prendre des lots de 20 solutions, générer doaine d'activité et description de ce qu'un utilisateur pourrait demander. En générer 3 ou 4 par solutions avec différents niveau de précision. Le faire pour une centaine de solutions.

On aurrait donc X = [description, domaine_activitée], y = [num_solution ].

Pour tester notre modèle on peut tenter de lui faire prédire les meilleures solutions en fonction de la description et du domaine_activitée. Si la solution correspondante est dans le top 3 des solutions prédites, on considère que c'est une réussite.

# 3. Modèles à tester


TF-IDF (c'est long et peu performant mais méthode sans embedding. On peut en parler dans Questionnaire et dire qu'on ne l'a pas implémenté car peu pertinent suivant les études qu'on a lu.)

FastText (on a un token par mot et on fait la moyenne pour avoir le texte)

Sentence transformers (un token par phrase et on fait la moyenne pour avoir le texte)

Doc2Vec (document lvl)
